# Введение в MapReduce модель на Python


In [48]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [49]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)
    
def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [50]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [51]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [52]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [53]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [54]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [55]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [56]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [57]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [58]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [59]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных. 

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [60]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*
 
mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*
 
mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL 

In [61]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str
    
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)
    
def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)
 
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication 

In [62]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])
 
def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])
      
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, np.float64(1.339523572430262)),
 (1, np.float64(1.339523572430262)),
 (2, np.float64(1.339523572430262)),
 (3, np.float64(1.339523572430262)),
 (4, np.float64(1.339523572430262))]

## Inverted index 

In [63]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)
      
def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)
 
def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('it', ['0', '1', '2']),
 ('is', ['0', '1', '2']),
 ('what', ['0', '1']),
 ('banana', ['2']),
 ('a', ['2'])]

## WordCount

In [64]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):  
    yield (word, 1)
 
def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [65]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()
      
def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]
 
def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers
  
def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)
  
  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*
 
e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*
 
flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount 

In [66]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps
  
  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)
      
  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):  
    yield (word, 1)
 
def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)
  
# try to set COMBINER=REDUCER and look at the number of values sent over the network 
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None) 
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('a', 2), ('is', 18), ('it', 18)]),
 (1, [('banana', 2), ('what', 10)])]

## TeraSort

In [67]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps
  
  def RECORDREADER(split):
    for value in split:
        yield (value, None)
      
  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])
    
def MAP(value:int, _):
  yield (value, None)
  
def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)
  
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, np.float64(0.0009001143906071096)),
   (None, np.float64(0.08076023297840706)),
   (None, np.float64(0.09411480341571854)),
   (None, np.float64(0.10629368930256866)),
   (None, np.float64(0.15082659970999335)),
   (None, np.float64(0.1550197468117116)),
   (None, np.float64(0.2799034348251481)),
   (None, np.float64(0.3342190145568843)),
   (None, np.float64(0.35706937748827816)),
   (None, np.float64(0.37093477865547897)),
   (None, np.float64(0.3853289788850601)),
   (None, np.float64(0.39499766262238645)),
   (None, np.float64(0.42572992877400384)),
   (None, np.float64(0.4485961151271435)),
   (None, np.float64(0.4860941917708774))]),
 (1,
  [(None, np.float64(0.5389807123674928)),
   (None, np.float64(0.560264159262098)),
   (None, np.float64(0.5619100375878651)),
   (None, np.float64(0.5897914761779816)),
   (None, np.float64(0.6167377666782948)),
   (None, np.float64(0.6781822480221741)),
   (None, np.float64(0.6835550583112384)),
   (None, np.float64(0.718267665

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [68]:
from typing import Iterator, NamedTuple
vec = [10, 2, 3, 4, 5]


def RECORDREADER():
    for value in vec:
        yield (value, None)
        
def MAP(value:int, _):
    yield ("max", value)
    
def REDUCE(key, nums:Iterator[NamedTuple]):
    max = -100000000
    for num in nums:
        if (num > max):
            max = num
    yield (key, max)


output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('max', 10)]

### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [69]:
vec = [10, 10, 9, 9]

def RECORDREADER():
    for value in vec:
        yield (value, None)

def MAP(value: int, _):
    yield ("avg", value)

def REDUCE(key: str, nums: Iterator[int]):
    total = 0
    count = 0
    for num in nums:
        total += num
        count += 1
    yield (key, total / count if count else 0.0)

output = list(MapReduce(RECORDREADER, MAP, REDUCE))
output

[('avg', 9.5)]

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [70]:
from itertools import groupby
from operator import itemgetter
from typing import Iterable, Iterator, Tuple, TypeVar, List

def groupbykey_sort_list(pairs):
    sorted_pairs = sorted(pairs, key=itemgetter(0))
    for k, group in groupby(sorted_pairs, key=itemgetter(0)):
        yield (k, [v for _, v in group])

arr = ((1, "k"), (2, "f"), (1, "d"), (1, "k"), (3, "n"), (2, "k"))
output = groupbykey_sort_list(arr)
for key, value in output:
    print(key,value)    

1 ['k', 'd', 'k']
2 ['f', 'k']
3 ['n']


### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [104]:
data = [1, 2, 2, 3, 3, 3, 10, 10, 5, 5, 5]

def INPUTFORMAT():
    parts = [data[:4], data[4:8], data[8:]]
    for part in parts:
        def record_reader(part=part):
            for t in part:
                yield (None, t)   
        yield record_reader()

def MAP(_, t):
    yield (t, 1)

def REDUCE(key, values: Iterator[Any]):
    yield key

def COMBINER(key, values: Iterator[Any]):
    yield (key, 1)

results = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=COMBINER)

distinct = list(flatten(vals for (_, vals) in results))         
print(sorted(distinct))  

6 key-value pairs were sent over a network.
[1, 2, 3, 5, 10]


#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [72]:
from typing import Any

data = [1, 2, 3, 4, 5, 6]

def predicate(t):
    return t == 4

def RECORDREADER():
    for t in data:
        yield (None, t)

def MAP(_, t):
    if predicate(t):
        yield (t, t)

def REDUCE(key, values: Iterator[Any]):
    for v in values:
        yield (key, v)

out = list(MapReduce(RECORDREADER, MAP, REDUCE))
print(out)

[(4, 4)]


### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [73]:
R = [
    (1, 3, 10),
    (1, 3, 20),
    (2, 4, 20),
    (2, 4, 20),  
]

L = [0, 1]

def RECORDREADER():
    for t in R:
        yield (None, t)

def MAP(_, t):
    t_proj = tuple(t[i] for i in L)   
    yield (t_proj, 1) 

def REDUCE(key, values: Iterator[Any]):
    for _ in values:
        yield key   
        break

out = list(MapReduce(RECORDREADER, MAP, REDUCE))
print(out)

[(1, 3), (2, 4)]


### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [74]:
from typing import Any

A = [1, 2, 3, 4]
B = [3, 4, 5, 6]

def RECORDREADER():
    for a in A:
        yield ("A", a)
    for b in B:
        yield ("B", b)

def MAP(_, t):
    yield (t, t)

def REDUCE(key, values: Iterator[Any]):
    for _ in values:
        yield key
        break
        
out = list(MapReduce(RECORDREADER, MAP, REDUCE))
print(out)

[1, 2, 3, 4, 5, 6]


### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [75]:
from typing import Any

A = [1, 2, 3, 4]
B = [3, 4, 5, 6]

def RECORDREADER():
    for a in A:
        yield ("A", a)
    for b in B:
        yield ("B", b)
        
def MAP(source, value):
    yield (value, source)

def REDUCE(key, values: Iterator[Any]):
    print(key, values)
    no_dublicates = set(values)
    if len(no_dublicates) == 2:
        yield key

out = list(MapReduce(RECORDREADER, MAP, REDUCE))
print(out)

1 ['A']
2 ['A']
3 ['A', 'B']
4 ['A', 'B']
5 ['B']
6 ['B']
[3, 4]


### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [76]:
from typing import Any

A = [1, 2, 3, 4]
B = [3, 4, 5, 6]

def RECORDREADER():
    for a in A:
        yield ("A", a)
    for b in B:
        yield ("B", b)
        
def MAP(source, value):
    yield (value, source)

def REDUCE(key, values: Iterator[Any]):
    sources = set(values)
    if sources == {"A"}:    
        yield key

out = list(MapReduce(RECORDREADER, MAP, REDUCE))
print(out)

[1, 2]


### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [77]:
A = [(1, 3), (2, 4), (7, 4)]     
B = [(3, 10), (4, 20), (4, 30)]

def RECORDREADER():
    for a,b in A:
        yield ("A", (a,b))
    for b,c in B:
        yield ("B", (b,c))
        
def MAP(source, value):
    if source == "A":
        a,b = value
        yield (b, ("A", a))
    if source == "B":
        b,c = value
        yield (b, ("B", c))

def REDUCE(key, values: Iterator[Any]):
    vec1 = []
    vec2 = []
    for tag, x in values:
        if tag == "A":
            vec1.append(x) 
        else:
            vec2.append(x)  

    for a in vec1:
        for c in vec2:
            yield (a, key, c)

out = list(MapReduce(RECORDREADER, MAP, REDUCE))
print(out)

[(1, 3, 10), (2, 4, 20), (2, 4, 30), (7, 4, 20), (7, 4, 30)]


### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [78]:
R = [
    ("u1", 10, "x"),
    ("u1", 7,  "y"),
    ("u2", 3,  "z"),
    ("u2", 9,  "w"),
    ("u2", 1,  "q"),
]

def RECORDREADER():
    for t in R:
        yield (None, t)

def theta(values: Iterator[float]) -> float:
    s = 0.0
    for v in values:
        s += v
    return s

def MAP(_, t: Tuple[Any, Any, Any]):
    a, b, c = t
    yield (a, b)   

def REDUCE(a, values: Iterator[Any]):
    x = theta(values)
    yield (a, x)

out = list(MapReduce(RECORDREADER, MAP, REDUCE))
print(out)

[('u1', 17.0), ('u2', 13.0)]


### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


In [105]:
mat = np.ones((5, 4))
vec = np.random.rand(4)  

def RECORDREADER1():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield (("M", i, j), float(mat[i, j]))

  for j in range(len(vec)):
    yield (("V", j), float(vec[j]))

def MAP1(k1, v1):
  tag = k1[0]
  if tag == "M":
    _, i, j = k1
    aij = v1
    # ключ = j, чтобы состыковать с v_j
    yield (j, ("M", i, aij))
  else:  # "V"
    _, j = k1
    vj = v1
    yield (j, ("V", vj))

def REDUCE1(j, values: Iterator[Any]):
  vj = None
  a_list = []  # (i, aij)

  for rec in values:
    if rec[0] == "V":
      _, vj = rec
    else:
      _, i, aij = rec
      a_list.append((i, aij))

  if vj is None:
    return  # если нет v_j , то умножать нечего

  for i, aij in a_list:
    yield (i, aij * vj)

partials = list(MapReduce(RECORDREADER1, MAP1, REDUCE1))

# суммирование по i
def RECORDREADER2():
  for i, contrib in partials:
    yield (i, contrib)

def MAP2(i, contrib):
  yield (i, contrib)

def REDUCE2(i, contribs: Iterator[float]):
  s = 0.0
  for x in contribs:
    s += x
  yield (i, s)

output = list(MapReduce(RECORDREADER2, MAP2, REDUCE2))
print(output)

y = np.array([v for (i, v) in sorted(output, key=lambda x: x[0])])
print("check:", np.allclose(y, mat @ vec))

[(0, 0.9889688176405107), (1, 0.9889688176405107), (2, 0.9889688176405107), (3, 0.9889688176405107), (4, 0.9889688176405107)]
check: True


## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$. 





In [79]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [88]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j,k), big_mat[j,k])
      
def MAP(k1, v1):
  (j, k) = k1
  w = v1
  for i in range(small_mat.shape[0]):
    yield ((i, k), small_mat[i, j] * w)
  # solution code that yield(k2,v2) pairs
  
def REDUCE(key, values):
  (i, k) = key
  s = 0.0
  for x in values:
    s += x
  yield ((i, k), s)
  # solution code that yield(k3,v3) pairs

Проверьте своё решение

In [89]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat) 
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [90]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [92]:
I, J, K = 2, 3, 4
A = np.random.rand(I, J)
B = np.random.rand(J, K)

# join по j 
def RECORDREADER1():
  for i in range(I):
    for j in range(J):
      yield (("A", i, j), float(A[i, j]))
  for j in range(J):
    for k in range(K):
      yield (("B", j, k), float(B[j, k]))

def MAP1(k1, v1):
  tag = k1[0]
  if tag == "A":
    _, i, j = k1
    aij = v1
    # ключ = j, значение хранит i и aij
    yield (j, ("A", i, aij))
  else:  # "B"
    _, j, k = k1
    bjk = v1
    # ключ = j, значение хранит k и bjk
    yield (j, ("B", k, bjk))

def REDUCE1(j, values: Iterator[Any]):
  a_list = []  # (i, aij)
  b_list = []  # (k, bjk)
  for rec in values:
    if rec[0] == "A":
      _, i, aij = rec
      a_list.append((i, aij))
    else:
      _, k, bjk = rec
      b_list.append((k, bjk))

  # для данного j создаём все пары i×k
  for i, aij in a_list:
    for k, bjk in b_list:
      yield ((i, k), aij * bjk)

partials = list(MapReduce(RECORDREADER1, MAP1, REDUCE1))

# суммирование по (i,k) 
def RECORDREADER2():
  for ik, contrib in partials:
    yield (ik, contrib)

def MAP2(k1, v1):
  # уже нужный ключ (i,k)
  yield (k1, v1)

def REDUCE2(ik, values: Iterator[float]):
  s = 0.0
  for x in values:
    s += x
  yield (ik, s)

reduce_output = list(MapReduce(RECORDREADER2, MAP2, REDUCE2))

def asmatrix(reduce_output):
  I2 = max(i for ((i, k), v) in reduce_output) + 1
  K2 = max(k for ((i, k), v) in reduce_output) + 1
  C = np.empty((I2, K2), dtype=float)
  for (i, k), v in reduce_output:
    C[i, k] = v
  return C

C_result = asmatrix(reduce_output)
C_reference = A @ B
print("allclose:", np.allclose(C_result, C_reference))

allclose: True


Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER. 

In [96]:
I = 2
J = 3
K = 4*10

small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)
reference_solution = np.matmul(small_mat, big_mat)

def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  
    if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)

  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)

  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output) + 1
  K = max(k for ((i,k), vw) in reduce_output) + 1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

# Генератор для вводных данных
def INPUTFORMAT():
  first_mat = []
  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      first_mat.append(((0, i, j), small_mat[i,j])) # первая матрица
  yield first_mat

  second_mat = []
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      second_mat.append(((1, j, k), big_mat[j,k])) # вторая матрица
  yield second_mat

# MAP функция для соединения матриц
def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1
  if mat_num == 0:
    yield (j, (mat_num, i, w))
  else:
    yield (i, (mat_num, j, w))

# REDUCE функция для соединения матриц
def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]
  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])

def GET_JOINED():
  for j in joined:
    print("aa", j)
    yield j[1]

def MAP_MUL(k1, v1):
  yield (k1, v1)

def REDUCE_MUL(key, values):
  res_val = 0
  for v in values:
    res_val += v
  yield (key, res_val)

maps = 4
reducers = 2

# Выполнение MapReduce для соединения матриц
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(joined)

# Выполнение MapReduce для умножения значений
mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)

solution = []
for p in pre_result:
  for v in p[1]:
    solution.append(v)

print(solution)
np.allclose(reference_solution, asmatrix(solution))

126 key-value pairs were sent over a network.
[(0, [((0, 0), np.float64(0.1263265334395539)), ((0, 1), np.float64(0.24789920736434531)), ((0, 2), np.float64(0.11157916858253988)), ((0, 3), np.float64(0.07315339828842952)), ((0, 4), np.float64(0.18778766802125832)), ((0, 5), np.float64(0.24957528028990372)), ((0, 6), np.float64(0.045598815905701834)), ((0, 7), np.float64(0.024530938216160644)), ((0, 8), np.float64(0.12930008645313115)), ((0, 9), np.float64(0.16234452270312727)), ((0, 10), np.float64(0.06926221278873303)), ((0, 11), np.float64(0.0707090086351858)), ((0, 12), np.float64(0.18907113409596155)), ((0, 13), np.float64(0.16347913902056002)), ((0, 14), np.float64(0.16215128477614918)), ((0, 15), np.float64(0.26279686728435353)), ((0, 16), np.float64(0.3040094432120828)), ((0, 17), np.float64(0.05522886786913401)), ((0, 18), np.float64(0.09303018790040071)), ((0, 19), np.float64(0.052451311354144305)), ((0, 20), np.float64(0.30831475906841377)), ((0, 21), np.float64(0.07550872529

True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [97]:
I = 2  
J = 3  
K = 4*10  

small_mat = np.random.rand(I, J)  
big_mat = np.random.rand(J, K)  
reference_solution = np.matmul(small_mat, big_mat)

def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())

  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)

  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)  

  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

# Функция для преобразования результата REDUCE в матрицу
def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output) + 1
  K = max(k for ((i,k), vw) in reduce_output) + 1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

# Генератор для вводных данных
def INPUTFORMAT():
  first_mat = []

  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      first_mat.append(((0, i, j), small_mat[i,j]))  # первая матрица

  global maps
  split_size = int(np.ceil(len(first_mat)/maps))

  for i in range(0, len(first_mat), split_size):
    yield first_mat[i:i+split_size]

  second_mat = []

  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      second_mat.append(((1, j, k), big_mat[j,k]))  # вторая матрица

  split_size = int(np.ceil(len(second_mat)/maps))

  for i in range(0, len(second_mat), split_size):
    yield second_mat[i:i+split_size]

# MAP функция для соединения матриц
def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1

  if mat_num == 0:
    yield (j, (mat_num, i, w))

  else:
    yield (i, (mat_num, j, w))

def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]

  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])

def GET_JOINED():

  for j in joined:
    print("aa", j)
    yield j[1]

def MAP_MUL(k1, v1):
  yield (k1, v1)

def REDUCE_MUL(key, values):
  res_val = 0

  for v in values:
    res_val += v
  yield (key, res_val)

maps = 3
reducers = 2

# Выполнение MapReduce для соединения матриц
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(joined)

# Выполнение MapReduce для умножения значений
mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)

solution = []

for p in pre_result:

  for v in p[1]:
    solution.append(v)

print(solution)

np.allclose(reference_solution, asmatrix(solution))

126 key-value pairs were sent over a network.
[(0, [((0, 0), np.float64(0.2941632424763946)), ((0, 1), np.float64(0.5406375237776415)), ((0, 2), np.float64(0.07982747251934827)), ((0, 3), np.float64(0.014013759520620199)), ((0, 4), np.float64(0.27522304978496603)), ((0, 5), np.float64(0.18903032478128523)), ((0, 6), np.float64(0.03645787802933058)), ((0, 7), np.float64(0.1767387723720886)), ((0, 8), np.float64(0.20181570379284633)), ((0, 9), np.float64(0.11212178698399015)), ((0, 10), np.float64(0.6881171560161139)), ((0, 11), np.float64(0.1287637644190757)), ((0, 12), np.float64(0.17610516941362517)), ((0, 13), np.float64(0.025002453156011944)), ((0, 14), np.float64(0.5192014182632879)), ((0, 15), np.float64(0.7257585123760033)), ((0, 16), np.float64(0.3069835796067584)), ((0, 17), np.float64(0.624150810835523)), ((0, 18), np.float64(0.5413347158004242)), ((0, 19), np.float64(0.44252248957430246)), ((0, 20), np.float64(0.43541788871770415)), ((0, 21), np.float64(0.4465235903135413)), 

True